## Preparation

### (1) Import packages

In [ ]:
import os
from typing import Tuple, List, Sequence, Callable

import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch

from torch import nn, Tensor
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset

!pip install -U git+https://github.com/albu/albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch.nn.functional as F
from sklearn.model_selection import train_test_split

import time

#추가설치
!pip install torch_optimizer
!pip install timm
!pip install -U git+https://github.com/lukemelas/EfficientNet-PyTorch
from efficientnet_pytorch import EfficientNet
import timm

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-wckn_j2u
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-wckn_j2u
  Created wheel for albumentations: filename=albumentations-1.0.0-cp37-none-any.whl size=98151 sha256=048f78938b9cfee0de958e9fb1f318e0f25cb9101c3545761dcfaeaaefddd00b
  Stored in directory: /tmp/pip-ephem-wheel-cache-khxju0db/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
Successfully built albumentations
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 348kB 14.4MB/s 
  Cloning https://github.com/lukemelas/EfficientNet-PyTorch to /tmp/pip-req-build-ds8jtdni
  Running command git clone -q https://github.com/lukemelas/EfficientNet-PyTorch /tmp/pip-req-build-ds8jtdni
  Created wheel for efficientnet-pytorc

### (2) Google Drive Colab 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/statml_competition/')

### (3) Setting

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

print('using device:', device)

using device: cuda:0


In [ ]:
device = "cuda:0"
dtype = torch.float
ltype = torch.long # entropy

In [ ]:
# fake 1, real 0 로 사용
train_df = pd.read_csv('./face_image/face_images.csv')
train_df.head()

,path,real,fake
0,./face_image/fake/JFH50GFJUL.jpg,0,1
1,./face_image/fake/0VPS5TI60G.jpg,0,1
2,./face_image/real/61911.jpg,1,0
3,./face_image/fake/APADHGXN31.jpg,0,1
4,./face_image/fake/SJO2UL69C2.jpg,0,1


In [ ]:
print(train_df.shape)

(20000, 3)


# random seed 고정
참고 : https://dacon.io/codeshare/2363?dtype=recent


In [ ]:
import random
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

## 커스텀 데이터 정의

In [ ]:
class FaceDataset(Dataset):
  def __init__(self, image_label, transforms) :
    self.df = image_label
    self.transforms = transforms
        
  def __len__(self) -> int:
    return self.df.shape[0]

  def __getitem__(self, index: int) -> Tuple[Tensor]:
    assert index <= len(self), 'index range error' 
      
    image_dir = self.df.iloc[index, ]['path']
    image_id = self.df.iloc[index, ]['fake'].astype(np.int64)
    
    image =  cv2.imread(image_dir, cv2.COLOR_BGR2RGB)
    target = torch.as_tensor(image_id, dtype=torch.long)

    if self.transforms is not None :
      image = self.transforms(image=image)['image']

    image = image/255.0
    
    return image, target

class TestDataset(Dataset):
  def __init__(self, image, transforms) :
    self.image = image
    self.transforms = transforms
        
  def __len__(self) -> int:
    return len(self.image)

  def __getitem__(self, index: int) -> Tuple[Tensor]:
    assert index <= len(self), 'index range error' 
    
    image_name = self.image[index]
    image_dir = './face_image/test_v1.1/' + image_name

    image =  cv2.imread(image_dir, cv2.COLOR_BGR2RGB)
    
    if self.transforms is not None :
      image = self.transforms(image=image)['image']
    image = image/255.0

    return image_name, image

## 이미지 어그멘테이션
( 참고 : https://github.com/albumentations-team/albumentations )

In [ ]:
transforms_tr = A.Compose([
    A.Resize(128, 128),
    ToTensorV2(), ])

transforms_val = A.Compose([
    A.Resize(128, 128),
    ToTensorV2(), ])

#Resnet 18-50 ensemble

참고 자료: https://deep-learning-study.tistory.com/534


In [ ]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        # BatchNorm에 bias가 포함되어 있으므로, conv2d는 bias=False로 설정합니다.
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion),
        )

        # identity mapping, input과 output의 feature map size, filter 수가 동일한 경우 사용.
        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        # projection mapping using 1x1conv
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x


class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels*BottleNeck.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels*BottleNeck.expansion)
            )
            
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=10, init_weights=True):
        super().__init__()

        self.in_channels=64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        #weights inittialization
        if init_weights:
            self._initialize_weights()

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self,x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    # define weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def resnet18():
    return ResNet(BasicBlock, [2,2,2,2])

def resnet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def resnet50():
    return ResNet(BottleNeck, [3,4,6,3])

def resnet101():
    return ResNet(BottleNeck, [3, 4, 23, 3])

def resnet152():
    return ResNet(BottleNeck, [3, 8, 36, 3])

In [ ]:
class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB, nb_classes=10):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
    
        # Remove last linear layer
        self.modelA.fc = nn.Identity()
        self.modelB.fc = nn.Identity()
        
        # Create new classifier
        self.classifier = nn.Linear(2048+512, nb_classes)
        
    def forward(self, x):
        x1 = self.modelA(x.clone())  # clone to make sure x is not changed by inplace methods
        x1 = x1.view(x1.size(0), -1)
        x2 = self.modelB(x)
        x2 = x2.view(x2.size(0), -1)
        x = torch.cat((x1, x2), dim=1)
        
        x = self.classifier(F.relu(x))
        return x

modelA = resnet18().to(device)
modelB = resnet50().to(device)

In [ ]:
num_epochs = 30
EARLY_STOPPING_EPOCH = 5
SEED=42

from sklearn.model_selection import KFold
kfold=KFold(n_splits=5, shuffle=True, random_state=SEED) #5-fold

resnet_models=[] #각 fold의 best model을 담을 리스트 

for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(train_df),1):
  print('[fold: %d]' % fold_index)

  torch.cuda.empty_cache()

  train=train_df.iloc[trn_idx]
  valid=train_df.iloc[val_idx]

  tr_dataset=FaceDataset(image_label=train, transforms=transforms_tr)
  val_dataset=FaceDataset(image_label=valid, transforms=transforms_val)

  train_loader=DataLoader(tr_dataset, batch_size=64, shuffle=True, num_workers=8)
  valid_loader=DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=8)

  #모델 정의
  model = MyEnsemble(modelA, modelB)
  model.to(device)

  optimizer = optim.Adam(model.parameters(), lr=7e-4, weight_decay=0) #l2 reg 적용
  lr_sched = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 

  valid_early_stop = 0
  valid_best_loss = float('inf')
  since = time.time()

  final_train_loss = []
  final_train_acc = []
  final_valid_loss = []
  final_valid_acc = []

  for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    model.train()
    for i, (images, targets) in enumerate(train_loader) : 
      optimizer.zero_grad()
      
      images = images.to(device, dtype)
      targets = targets.to(device, ltype)
    
      scores = model(images)
      _, preds = scores.max(dim=1)

      loss = F.cross_entropy(scores, targets)
      loss.backward()
      optimizer.step()
      
      correct = sum(targets == preds).cpu()
      acc=(correct/64 * 100)

      train_loss_list.append(loss)
      train_acc_list.append(acc)

      if i % 20 == 0 :
        print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)
    
    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    model.eval()
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(valid_loader) : 
      optimizer.zero_grad()
      images = images.to(device=device, dtype=dtype)
      targets = targets.to(device=device, dtype=ltype)

      with torch.no_grad():
        scores = model(images)
        loss = F.cross_entropy(scores, targets)
        _, preds = scores.max(dim=1)
      
      correct = sum(targets == preds).cpu()
      acc=(correct/64 * 100)

      valid_loss_list.append(loss)
      valid_acc_list.append(acc)
  
    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
      valid_best_loss = val_mean_loss
      valid_early_stop = 0
      # new best model save (valid 기준)
      best_model = model
      if (e+1) == num_epochs:
        resnet_models.append(best_model)
      

    else:
      # early stopping    
      valid_early_stop += 1
      if valid_early_stop >= EARLY_STOPPING_EPOCH:
        print("EARLY STOPPING!!")
        resnet_models.append(best_model)
        break

    lr_sched.step()

[fold: 1]
 ====================== epoch 1 ======================


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Iteration   0 | Train Loss  2.1438 | Classifier Accuracy 12.50
Iteration  20 | Train Loss  0.6735 | Classifier Accuracy 57.81
Iteration  40 | Train Loss  0.4896 | Classifier Accuracy 75.00
Iteration  60 | Train Loss  0.5552 | Classifier Accuracy 70.31
Iteration  80 | Train Loss  0.6521 | Classifier Accuracy 64.06
Iteration 100 | Train Loss  0.5947 | Classifier Accuracy 71.88
Iteration 120 | Train Loss  0.3955 | Classifier Accuracy 75.00
Iteration 140 | Train Loss  0.6125 | Classifier Accuracy 71.88
Iteration 160 | Train Loss  0.4911 | Classifier Accuracy 78.12
Iteration 180 | Train Loss  0.4794 | Classifier Accuracy 75.00
Iteration 200 | Train Loss  0.5912 | Classifier Accuracy 71.88
Iteration 220 | Train Loss  0.4456 | Classifier Accuracy 81.25
Iteration 240 | Train Loss  0.4918 | Classifier Accuracy 79.69

[Summary] Elapsed time : 16 m 18 s
Train Loss Mean 0.6232 | Accuracy 68.89 
Valid Loss Mean 0.5230 | Accuracy 73.19 

 ====================== epoch 2 ======================
Iterati

# EfficientNet B7

In [ ]:
class efficientnet(torch.nn.Module):
  def __init__(self):
    super(efficientnet, self).__init__()
    self.conv2d = nn.Conv2d(3, 3, 3, stride=1)
    self.effnet = EfficientNet.from_name('efficientnet-b7',dropout_rate=0.2) 
    self.FC = nn.Linear(1000,2)

    nn.init.xavier_normal_(self.FC.weight)

  def forward(self, x):
    x = F.silu(self.conv2d(x))

    #effnet추가
    x = F.silu(self.effnet(x))
    
    x = self.FC(x)
    return x

model = efficientnet()

In [ ]:
#optimizer 설정
import torch_optimizer as optim
model.to(device)
optimizer = optim.RAdam(model.parameters(), lr=0.0015, betas=(0.9,0.999), weight_decay=0)
lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

In [ ]:
#모델 훈련
num_epochs = 40
EARLY_STOPPING_EPOCH = 7
SEED = 42

# cross validation을 적용하기 위한 KFold 생성
from sklearn.model_selection import KFold
kfold = KFold(n_splits=4, shuffle=True, random_state=SEED) #4-fold

effi_models=[] #각 fold의 best model을 담을 리스트 

#train_df에서 train_index와 valid_index 생성
for fold_index, (train_index, valid_index) in enumerate(kfold.split(train_df),1): #enumerate(,1) 1부터 index시작
  print('[fold: %d' %fold_index)

  #CUDA cahe 초기화
  torch.cuda.empty_cache()

  #train fold, valid fold 분할
  train = train_df.iloc[train_index]
  valid = train_df.iloc[valid_index]

  tr_dataset = FaceDataset(image_label=train, transforms=transforms_tr)
  val_dataset = FaceDataset(image_label=valid, transforms=transforms_val)

  train_loader = DataLoader(tr_dataset, batch_size=64, shuffle=True, num_workers=8)
  valid_loader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=8)

  #train 시작
  valid_early_stop = 0

  valid_best_loss = float('inf')
  since = time.time()

  final_train_loss = []
  final_train_acc = []
  final_valid_loss = []
  final_valid_acc = []

  for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    model.train()
    for i, (images, targets) in enumerate(train_loader) : 
      optimizer.zero_grad()
      
      images = images.to(device, dtype)
      targets = targets.to(device, ltype)
    
      scores = model(images)
      _, preds = scores.max(dim=1)

      loss = F.cross_entropy(scores, targets)
      loss.backward()
      optimizer.step()
      
      correct = sum(targets == preds).cpu()
      acc=(correct/64 * 100)

      train_loss_list.append(loss)
      train_acc_list.append(acc)

      if i % 20 == 0 :
        print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)
    
    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    model.eval()
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(valid_loader) : 
      optimizer.zero_grad()
      images = images.to(device=device, dtype=dtype)
      targets = targets.to(device=device, dtype=ltype)

      with torch.no_grad():
        scores = model(images)
        loss = torch.nn.functional.cross_entropy(scores, targets)
        _, preds = scores.max(dim=1)
      
      correct = sum(targets == preds).cpu()
      acc=(correct/64 * 100)

      valid_loss_list.append(loss)
      valid_acc_list.append(acc)
  
    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
      valid_best_loss = val_mean_loss
      valid_early_stop = 0
      best_model = model
      if (e+1) == num_epochs:
        effi_models.append(best_model)
        
    else:
      # early stopping    
      valid_early_stop += 1
      if valid_early_stop >= EARLY_STOPPING_EPOCH:
        print("EARLY STOPPING!!")
        effi_models.append(best_model)
        break

    lr_sched.step()

[fold: 1
 ====================== epoch 1 ======================


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Iteration   0 | Train Loss  0.7392 | Classifier Accuracy 43.75
Iteration  20 | Train Loss  0.7423 | Classifier Accuracy 50.00
Iteration  40 | Train Loss  0.7831 | Classifier Accuracy 50.00
Iteration  60 | Train Loss  0.6495 | Classifier Accuracy 56.25
Iteration  80 | Train Loss  0.6801 | Classifier Accuracy 56.25
Iteration 100 | Train Loss  0.7611 | Classifier Accuracy 53.12
Iteration 120 | Train Loss  0.7058 | Classifier Accuracy 53.12
Iteration 140 | Train Loss  0.6762 | Classifier Accuracy 62.50
Iteration 160 | Train Loss  0.7137 | Classifier Accuracy 60.94
Iteration 180 | Train Loss  0.7609 | Classifier Accuracy 48.44
Iteration 200 | Train Loss  0.6428 | Classifier Accuracy 51.56
Iteration 220 | Train Loss  0.6774 | Classifier Accuracy 54.69

[Summary] Elapsed time : 1 m 36 s
Train Loss Mean 0.7347 | Accuracy 52.22 
Valid Loss Mean 0.7142 | Accuracy 49.74 

 ====================== epoch 2 ======================
Iteration   0 | Train Loss  0.7256 | Classifier Accuracy 51.56
Iteratio

KeyboardInterrupt: ignored

# XceptionNet

In [ ]:
 #모델 훈련
num_epochs = 40
EARLY_STOPPING_EPOCH = 7
SEED = 42

# cross validation을 적용하기 위한 KFold 생성
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=SEED) #5-fold

xcep_models=[]

#train_df에서 train_index와 valid_index 생성
for fold_index, (train_index, valid_index) in enumerate(kfold.split(train_df),1): #enumerate(,1) 1부터 index시작
  print('[fold: %d' %fold_index)

  #CUDA cahe 초기화
  torch.cuda.empty_cache()

  #train fold, valid fold 분할
  train = train_df.iloc[train_index]
  valid = train_df.iloc[valid_index]

  tr_dataset = FaceDataset(image_label=train, transforms=transforms_tr)
  val_dataset = FaceDataset(image_label=valid, transforms=transforms_val)

  train_loader = DataLoader(tr_dataset, batch_size=64, shuffle=True, num_workers=8)
  valid_loader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=8)

  model = timm.create_model('xception', pretrained=False)
  model.to(device)
  optimizer = optim.RAdam(model.parameters(), lr=0.0015, betas=(0.9,0.999), weight_decay=0)
  lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

  #train 시작
  valid_early_stop = 0

  valid_best_loss = float('inf')
  since = time.time()

  final_train_loss = []
  final_train_acc = []
  final_valid_loss = []
  final_valid_acc = []

  for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    model.train()
    for i, (images, targets) in enumerate(train_loader) : 
      optimizer.zero_grad()
      
      images = images.to(device, dtype)
      targets = targets.to(device, ltype)
    
      scores = model(images)
      _, preds = scores.max(dim=1)

      loss = F.cross_entropy(scores, targets)
      loss.backward()
      optimizer.step()
      
      correct = sum(targets == preds).cpu()
      acc=(correct/64 * 100)

      train_loss_list.append(loss)
      train_acc_list.append(acc)

      if i % 20 == 0 :
        print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)
    
    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    model.eval()
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(valid_loader) : 
      optimizer.zero_grad()
      images = images.to(device=device, dtype=dtype)
      targets = targets.to(device=device, dtype=ltype)

      with torch.no_grad():
        scores = model(images)
        loss = torch.nn.functional.cross_entropy(scores, targets)
        _, preds = scores.max(dim=1)
      
      correct = sum(targets == preds).cpu()
      acc=(correct/64 * 100)

      valid_loss_list.append(loss)
      valid_acc_list.append(acc)
  
    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
      valid_best_loss = val_mean_loss
      valid_early_stop = 0
      best_model = model
      if (e+1) == num_epochs:
        xcep_models.append(best_model)

    else:
      # early stopping    
      valid_early_stop += 1
      if valid_early_stop >= EARLY_STOPPING_EPOCH:
        print("EARLY STOPPING!!")
        xcep_models.append(best_model)
        break

    lr_sched.step()

## 추론

In [ ]:
submission = pd.read_csv("./face_image/submission_v1.1.csv")
submission.head()

In [ ]:
test_dataset = TestDataset(submission['image'], transforms_val)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8) #num_workers 2에서 8로 변경

# SoftMax

In [ ]:
import numpy as np

def softmax_0(score0, score1):
  exp_a0 = np.exp(score0)
  exp_a1 = np.exp(score1)
  sum_exp=exp_a0+exp_a1
  y=exp_a0 / sum_exp
  return y

In [ ]:
scores_result=[]
prob_zero=[0 for _ in range(5000)]

model_list=[xcep_models[0],xcep_models[1],xcep_models[2],xcep_models[3],xcep_models[4],
            resnet_models[0],resnet_models[1],resnet_models[2],resnet_models[3],resnet_models[4],
            effi_models[0], effi_models[1], effi_models[2],effi_models[3]]

for i in range(14): #model 개수만큼 for문 돌리기
  predictions = []
  files = []
  score_list=[]

  with torch.no_grad():
    for img_names, images in test_loader:
      images = images.to(device=device, dtype=dtype)
      scores = model_list[i](images)
      _, preds = scores.max(dim=1)
      
      files.extend(img_names)
      predictions.extend(preds.squeeze(0).detach().cpu().numpy())
      score_list.extend(scores.squeeze(0).detach().cpu().numpy())

  for i, score in enumerate(score_list):
    prob_zero[i]+=softmax_0(score[0],score[1])

In [ ]:
for i in range(5000):
  prob_zero[i]/=14  #model 개수로 나눈다
  prob_zero[i]=(prob_zero[i]<0.5)*1

In [ ]:
df = pd.DataFrame(columns=submission.columns)
df['image'] = files
df['label'] = prob_zero
df.head(30)

In [ ]:
df.to_csv("./submission_group5.csv", index=False)